# **1. Data Loading**

In [ ]:
!pip install simpletransformers

import pandas as pd
from sklearn.model_selection import train_test_split


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

data = pd.read_excel('/content/updated_news_sentiment_analysis.xlsx', engine='openpyxl')

print(data.info())
print(data['Sentiment'].value_counts())

train_data, val_data = train_test_split(data, test_size=0.2, random_state=42)

print(f"Training set size: {len(train_data)}, Validation set size: {len(val_data)}")


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1725 entries, 0 to 1724
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Description  1725 non-null   object
 1   Sentiment    1725 non-null   object
dtypes: object(2)
memory usage: 27.1+ KB
None
Sentiment
negative    575
neutral     575
positive    575
Name: count, dtype: int64
Training set size: 1380, Validation set size: 345


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

data = pd.read_excel('/content/updated_news_sentiment_analysis.xlsx', engine='openpyxl')

train_data, val_data = train_test_split(data, test_size=0.2, random_state=42)

train_df = pd.DataFrame({
    'text': train_data['Description'],
    'labels': train_data['Sentiment']
})

val_df = pd.DataFrame({
    'text': val_data['Description'],
    'labels': val_data['Sentiment']
})

print("Before mapping - unique labels in train_df:", train_df['labels'].unique())
print("Before mapping - unique labels in val_df:", val_df['labels'].unique())

label_mapping = {"negative": 0, "positive": 1}  # Adjust mapping as needed

train_df['labels'] = train_df['labels'].map(label_mapping)
val_df['labels'] = val_df['labels'].map(label_mapping)

print("After mapping - unique labels in train_df:", train_df['labels'].unique())
print("After mapping - unique labels in val_df:", val_df['labels'].unique())

train_df = train_df.dropna(subset=['labels'])
val_df = val_df.dropna(subset=['labels'])

train_df['labels'] = train_df['labels'].astype(int)
val_df['labels'] = val_df['labels'].astype(int)

print("Sample of training data:")
print(train_df.head())

print("\nSample of validation data:")
print(val_df.head())


Before mapping - unique labels in train_df: ['negative' 'neutral' 'positive']
Before mapping - unique labels in val_df: ['neutral' 'negative' 'positive']
After mapping - unique labels in train_df: [ 0. nan  1.]
After mapping - unique labels in val_df: [nan  0.  1.]
Sample of training data:
                                                   text  labels
107   Justin Fessler currently serves as vice presid...       0
1706  US Bancorp DE reduced its stake in Zebra Techn...       1
704   The man who tried to assassinate Trump had pho...       0
677   Airmen assigned to the 92nd Air Refueling Wing...       0
916   Residents and visitors of Roquetas del Mar hav...       1

Sample of validation data:
                                                   text  labels
300   Delta Air Lines CEO Ed Bastian had stark words...       0
1510  PHILADELPHIA, July 18, 2024 /PRNewswire/ -- Wi...       0
1293  Oklahoma Attorney General Gentner Drummond on ...       1
23    World Wrestling Entertainment is in

# **2. Text PreProcessing**

In [ ]:
import re

def clean_health_text(text):
    text = text.lower()
    text = re.sub(r'[^\w\s\d/.-]', '', text)  # Keep letters, numbers, spaces, slashes, dots, and hyphens

    text = re.sub(r'\s+', ' ', text)
    text = text.strip()

    return text

train_df['text'] = train_df['text'].apply(clean_health_text)
val_df['text'] = val_df['text'].apply(clean_health_text)
print("Sample of cleaned training data:")
print(train_df.head())

Sample of cleaned training data:
                                                   text  labels
107   justin fessler currently serves as vice presid...       0
1706  us bancorp de reduced its stake in zebra techn...       1
704   the man who tried to assassinate trump had pho...       0
677   airmen assigned to the 92nd air refueling wing...       0
916   residents and visitors of roquetas del mar hav...       1


# **3. Text Embedding using BERT and RoBERTa**

**Training the BERT model**

In [ ]:
from simpletransformers.classification import ClassificationModel
training_args = {
    "num_train_epochs": 5,          # More epochs for better convergence
    "train_batch_size": 64,         # Larger batch size for stable gradients
    "eval_batch_size": 64,          # Match for evaluation
    "max_seq_length": 256,          # Longer sequences to capture full context
    "learning_rate": 2e-5,          # Slightly lower LR for finer updates
    "fp16": True,                   # Mixed precision to fit larger batches
    "overwrite_output_dir": True,   # Overwrite previous outputs
    "evaluate_during_training": True,  # Monitor validation performance
    "use_early_stopping": True,     # Stop if no improvement
    "early_stopping_patience": 2,   # Wait 2 epochs for improvement
    "early_stopping_delta": 0.01,   # Minimum improvement to count
    "adam_epsilon": 1e-8,           # Default optimizer stability
    "warmup_steps": 100,            # Gradual LR warmup for stability
    "weight_decay": 0.01,           # Regularization to prevent overfitting
    "output_dir": "outputs/",       # Temporary local directory
    "best_model_dir": "outputs/best_model/"  # Save best model
}

print("Training DistilBERT model...")
bert_model = ClassificationModel(
    "distilbert",
    "distilbert-base-uncased",
    num_labels=2,
    args=training_args,
    use_cuda = False
)

bert_model.train_model(train_df, eval_df=val_df)

Training DistilBERT model...


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


  0%|          | 0/1 [00:00<?, ?it/s]

Epoch:   0%|          | 0/5 [00:00<?, ?it/s]

Running Epoch 1 of 5:   0%|          | 0/15 [00:00<?, ?it/s]

**Training the Roberta Model**

In [ ]:
print("Training RoBERTa model...")
roberta_model = ClassificationModel(
    "roberta",
    "roberta-base",
    num_labels=2,
    args=training_args,
    use_cuda = True# Apply fast settings
)

roberta_model.train_model(train_df, eval_df=val_df)
# import torch
# torch.cuda.is_available()

# **5. Evaluate on Validation Set**

**Evaluate the BERT model**

In [ ]:
print("Evaluating DistilBERT model...")
result_bert, model_outputs_bert, wrong_predictions_bert = bert_model.eval_model(val_df)

print("DistilBERT Evaluation Results:")
print(result_bert)

**Evaluate the ROBERTA model**

In [ ]:
print("Evaluating RoBERTa model...")
result_roberta, model_outputs_roberta, wrong_predictions_roberta = roberta_model.eval_model(val_df)

print("RoBERTa Evaluation Results:")
print(result_roberta)

# **6. Saving the Best Model**

In [ ]:
bert_model.save_model('bert_best_model')

In [ ]:
roberta_model.save_model('roberta_best_model')

# **7. Prediction on Real-World Input**

In [ ]:
real_world_text = [
    "Stock markets rally as economic recovery gains momentum.",
    "Tech companies face major setbacks after regulatory crackdown.",
    "New environmental policies receive mixed reactions from the public.",
    "Oil prices remain stable despite global uncertainty.",
    "Major cybersecurity breach exposes personal data of millions."
]

# Predict the sentiment using DistilBERT
predictions_bert, _ = bert_model.predict(real_world_text)

label_mapping = {0: "negative", 1: "neutral", 2: "positive"}
predicted_labels = [label_mapping[p] for p in predictions_bert]

for text, label in zip(real_world_text, predicted_labels):
    print(f"News: {text}\nPredicted Sentiment: {label}\n")



In [ ]:
predictions_roberta, _ = roberta_model.predict(real_world_text)

label_mapping = {0: "negative", 1: "neutral", 2: "positive"}

print(f"RoBERTa Predictions: {predictions_roberta} (0 = negative, 1 = neutral, 2 = positive)")

